In [28]:
from torch import nn
import torch
import math

In [69]:
#WORKS NOW- Normalized over mean
#may want to take a look at other options of how to do this, but this has worked for others
#may also want to try positional encoding

class URLRet(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, num_layers, dropout):
        super(URLRet, self).__init__()
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=dim_feedforward)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        self.f1 = nn.Linear(d_model, 1024)
        self.f2 = nn.Linear(1024, 768)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.flatten = nn.Flatten()

    def forward(self, x):
        
        x = self.encoder(x)
        x = x.mean(dim = 1)
        x = self.flatten(x)

        x = self.f1(x)
        x = self.relu(x)
        x = self.f2(x)
        x = self.sigmoid(x)
        return x

d_model = 768
nhead = 4
dim_feedforward = 2048
num_layers=2
dropout = .1

model = URLRet(d_model, nhead, dim_feedforward, num_layers, dropout)
model = model.to('cuda:0')

In [70]:
learning_rate = 1e-1
loss_fn = nn.CosineEmbeddingLoss(margin=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [71]:
import sys
sys.path.insert(1, '../dataloaders')
from basic_dl import RedditDatasetFromPickle

In [72]:
rd = RedditDatasetFromPickle('../../data/reddit/bbc_news_scrape_raw.pkl')

In [74]:
model(rd[4][0].reshape(1, -1, 768)).shape

torch.Size([1, 768])

In [75]:
rd[0][1].shape

torch.Size([768])

In [84]:
import random
def train(rd, model):
    total_loss = 0
    model.train()
    for i in range(len(rd)):
        (x, y) = rd[i]
        x = x.reshape(1, -1, 768)
        pred = model(x)
        condition = torch.tensor(1).to('cuda:0')
        loss = loss_fn(pred.flatten(), y, condition)

        # randomly sample negative examples
        # should really do contrastive loss over the batch
        for i in range(5):
            neg_idx = random.randint(0,len(rd))
            if neg_idx == i: continue

            x_neg, y_neg = rd[i]
            x_neg = x_neg.reshape(1, -1, 768)

            pred = model(x_neg)

            condition = torch.tensor(0).to('cuda:0')
            loss_neg = loss_fn(pred.flatten(), y_neg, condition)

            loss += loss_neg
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    return total_loss / len(X_train)

In [ ]:
for epoch in range(100):
    loss = train(rd, model)
    print(loss)

In [24]:
rd[1][0].shape

torch.Size([9, 768])

In [51]:
pe = PositionalEncoding(1, .1).cuda()

In [57]:
pe(rd[0][0].reshape(-1, 1, 768)).shape

torch.Size([3, 1, 768])

In [54]:
rd[0][0].shape

torch.Size([3, 768])

In [56]:
pe(rd[1][0]).shape

torch.Size([9, 9, 768])

In [67]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
src = torch.rand(10, 32, 512)
out = transformer_encoder(src)

In [68]:
out.shape

torch.Size([10, 32, 512])